# Register Meshes to the First Mesh of the Time-Series

This notebook transforms (i.e. registers, or aligns) the meshes of the time-series to the first mesh using: translations (to center) and rotations only (to orient).

We will only keep the centered meshes, because the registration in rotation fails due to the fact that meshes significantly change shapes.

The meshes in the .ply files can be opened with:
- the `vscode-3d-preview` extension of VSCode
- [MeshLab](https://www.meshlab.net/)
- [Blender](https://www.blender.org/download/), specifically with its [Stop-motion-OBJ](https://github.com/neverhood311/Stop-motion-OBJ) plugin to visualize time-series of .ply files.

**Note:** Add a plugin to blender following [these instructions](https://docs.blender.org/manual/en/latest/editors/preferences/addons.html#rd-party-add-ons). 

## Set-up

In [20]:
import os
import subprocess

gitroot_path = subprocess.check_output(
    ["git", "rev-parse", "--show-toplevel"], universal_newlines=True
)
os.chdir(gitroot_path[:-1])
print("Working directory: ", os.getcwd())

import warnings

warnings.filterwarnings("ignore")

import sys

sys_dir = os.path.dirname(os.getcwd())
sys.path.append(sys_dir)
print("Directory added to path: ", sys_dir)

Working directory:  /home/adele/code/my28brains
Directory added to path:  /home/adele/code


## Imports

In [21]:
import glob

import numpy as np
import trimesh

import my28brains.io as io

MESHES_DIR = os.path.join(os.getcwd(), "data", "meshes")
CENTERED_MESHES_DIR = os.path.join(os.getcwd(), "data", "centered_meshes")
REGISTERED_MESHES_DIR = os.path.join(os.getcwd(), "data", "registered_meshes")
print("MESHES_DIR: ", MESHES_DIR)
print("CENTERED_MESHES_DIR: ", CENTERED_MESHES_DIR)
print("REGISTERED_MESHES_DIR: ", REGISTERED_MESHES_DIR)

MESHES_DIR:  /home/adele/code/my28brains/data/meshes
CENTERED_MESHES_DIR:  /home/adele/code/my28brains/data/centered_meshes
REGISTERED_MESHES_DIR:  /home/adele/code/my28brains/data/registered_meshes


## Functions to Register Meshes

In [22]:
def center_whole_brain(mesh):
    """Center a mesh by putting its barycenter at origin of the coordinates.

    Parameters
    ----------
    mesh : trimesh.Trimesh
        Mesh to center.

    Returns
    -------
    centered_mesh : trimesh.Trimesh
        Centered Mesh.
    brain_center: coordinates of center of the mesh before centering
    """
    vertices = mesh.vertices
    brain_center = np.mean(vertices, axis=0)
    centered_vertices = vertices - brain_center
    return trimesh.Trimesh(vertices=centered_vertices, faces=mesh.faces), brain_center

def center_substructure(mesh,brain_center):
    """Center a mesh by putting its barycenter at origin of the coordinates.

    Parameters
    ----------
    mesh : trimesh.Trimesh
        Mesh to center.
    center: coordinates of the vector that you would like to use to center your mesh
        i.e. in this case, this is the center of the whole brain, and mesh will
        be the meshes of the substructures of the brain.

    Returns
    -------
    centered_mesh : trimesh.Trimesh
        Centered Mesh.
    """
    vertices = mesh.vertices
    centered_vertices = vertices - brain_center
    return trimesh.Trimesh(vertices=centered_vertices, faces=mesh.faces)


def register_mesh(mesh, base_mesh):
    """Register a mesh to a base mesh.

    Note that the rigid registration slightly un-centered the registered mesh.

    Parameters
    ----------
    mesh : trimesh.Trimesh
        Mesh to register.

    Returns
    -------
    registered_mesh : trimesh.Trimesh
        Registered Mesh.
    """
    transform_mesh_to_base_mesh, cost = trimesh.registration.mesh_other(
        mesh=mesh, other=base_mesh, scale=False
    )
    print(f"Cost: {cost}")
    # Note: This modifies the original mesh in place
    registered_mesh = mesh.apply_transform(transform_mesh_to_base_mesh)
    return registered_mesh

In [23]:
def write_total_brain_centered_mesh(hemisphere, structure_id):
    string_base = os.path.join(
        MESHES_DIR, f"{hemisphere}_structure_{structure_id}**.ply"
    )
    paths = sorted(glob.glob(string_base))

    print(
        f"Found {len(paths)} ply files for {hemisphere} hemisphere and anatomical structure {structure_id}:\n {paths}\n"
    )

    # print(f"-> Registration: Using the first mesh as the base mesh.")
    # base_mesh = trimesh.load(paths[0])
    # centered_base_mesh = center_mesh(base_mesh)
    # ply_path = os.path.join(CENTERED_MESHES_DIR, os.path.basename(paths[0]))
    # io.write_trimesh_to_ply(mesh=centered_base_mesh, ply_path=ply_path)
    # ply_path = os.path.join(REGISTERED_MESHES_DIR, os.path.basename(paths[0]))
    # io.write_trimesh_to_ply(mesh=centered_base_mesh, ply_path=ply_path)

    brain_centers = []
    # for path in paths[1:]:
    for path in paths:
        print(f"\tLoad mesh from path: {path}")
        mesh = trimesh.load(path)
        centered_mesh, brain_center = center_whole_brain(mesh)
        brain_centers.append(brain_center)
        ply_path = os.path.join(CENTERED_MESHES_DIR, os.path.basename(path))
        io.write_trimesh_to_ply(mesh=centered_mesh, ply_path=ply_path)
    brain_centers = np.asarray(brain_centers)
    return brain_centers

In [24]:
def write_substructure_centered_mesh(hemisphere, structure_id, brain_centers):
    string_base = os.path.join(
        MESHES_DIR, f"{hemisphere}_structure_{structure_id}**.ply"
    )
    paths = sorted(glob.glob(string_base))

    print(
        f"Found {len(paths)} ply files for {hemisphere} hemisphere and anatomical structure {structure_id}:\n {paths}\n"
    )

    # print(f"-> Registration: Using the first mesh as the base mesh.")
    # base_mesh = trimesh.load(paths[0])
    # centered_base_mesh = center_mesh(base_mesh)
    # ply_path = os.path.join(CENTERED_MESHES_DIR, os.path.basename(paths[0]))
    # io.write_trimesh_to_ply(mesh=centered_base_mesh, ply_path=ply_path)
    # ply_path = os.path.join(REGISTERED_MESHES_DIR, os.path.basename(paths[0]))
    # io.write_trimesh_to_ply(mesh=centered_base_mesh, ply_path=ply_path)

    # for i_day, path in enumerate(paths[1:]):
    for i_day, path in enumerate(paths):
        print(f"\tLoad mesh from path: {path}")
        mesh = trimesh.load(path)
        centered_mesh = center_substructure(mesh, brain_centers[i_day])
        ply_path = os.path.join(CENTERED_MESHES_DIR, os.path.basename(path))
        io.write_trimesh_to_ply(mesh=centered_mesh, ply_path=ply_path)

## Left Hemisphere: Register Meshes on the First (Base) Mesh of the Time-Series

This takes a couple of minutes.

In [27]:
substructure_structure_ids = list(range(1,10))
# structure_ids.append(-1)

hemisphere = "left"

#first, center the "total brain" (shape id -1), and then use the centering vector for that to center the
# rest of the structures
brain_centers_left = write_total_brain_centered_mesh(hemisphere, structure_id = -1)

#now, use brain_center to center the rest of the meshes.
for structure_id in substructure_structure_ids:
    write_substructure_centered_mesh(hemisphere, structure_id, brain_centers_left)

Found 60 ply files for left hemisphere and anatomical structure -1:
 ['/home/adele/code/my28brains/data/meshes/left_structure_-1_day01.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day02.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day03.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day04.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day05.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day06.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day07.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day08.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day09.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day10.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day11.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day12.ply', '/home/adele/code/my28brains/data/meshes/left_structure_-1_day13.ply', '/home/

In [28]:
# string_base = os.path.join(
#         MESHES_DIR, f"{hemisphere}_structure_{structure_id}**.ply"
#     )
#     paths = sorted(glob.glob(string_base))

#     print(
#         f"Found {len(paths)} ply files for {hemisphere} hemisphere and anatomical structure {structure_id}:\n {paths}\n"
#     )

#     print(f"-> Registration: Using the first mesh as the base mesh.")
#     base_mesh = trimesh.load(paths[0])
#     centered_base_mesh = center_mesh(base_mesh)
#     ply_path = os.path.join(CENTERED_MESHES_DIR, os.path.basename(paths[0]))
#     io.write_trimesh_to_ply(mesh=centered_base_mesh, ply_path=ply_path)
#     # ply_path = os.path.join(REGISTERED_MESHES_DIR, os.path.basename(paths[0]))
#     # io.write_trimesh_to_ply(mesh=centered_base_mesh, ply_path=ply_path)

#     for path in paths[1:]:
#         print(f"\tLoad mesh from path: {path}")
#         mesh = trimesh.load(path)
#         centered_mesh = center_mesh(mesh)
#         ply_path = os.path.join(CENTERED_MESHES_DIR, os.path.basename(path))
#         io.write_trimesh_to_ply(mesh=centered_mesh, ply_path=ply_path)

#         # registered_mesh = register_mesh(mesh=centered_mesh, base_mesh=centered_base_mesh)

#         # ply_path = os.path.join(REGISTERED_MESHES_DIR, os.path.basename(path))
#         # io.write_trimesh_to_ply(mesh=registered_mesh, ply_path=ply_path)

## Right Hemisphere: Register Meshes on the First (Base) Mesh of the Time-Series

This takes a couple of minutes.

In [31]:
substructure_structure_ids = list(range(1,10))
# structure_ids.append(-1)

hemisphere = "right"

#first, center the "total brain" (shape id -1), and then use the centering vector for that to center the
# rest of the structures
brain_centers_right = write_total_brain_centered_mesh(hemisphere, structure_id = -1)

#now, use brain_center to center the rest of the meshes.
for structure_id in substructure_structure_ids:
    write_substructure_centered_mesh(hemisphere, structure_id, brain_centers_left)

Found 58 ply files for right hemisphere and anatomical structure -1:
 ['/home/adele/code/my28brains/data/meshes/right_structure_-1_day01.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day02.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day03.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day04.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day05.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day06.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day07.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day08.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day09.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day10.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day11.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day12.ply', '/home/adele/code/my28brains/data/meshes/right_structure_-1_day13